<a href="https://colab.research.google.com/github/yujinc129-oss/Drama_data_project/blob/main/drama_pre(1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np

# 배우 데이터셋 만들기

## 원본 데이터 확인

In [ ]:
actors_2 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/data_set/드라마 정보/data/mydramalistactors_tocsv.csv')
actors_3 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/data_set/드라마 정보/data/wiki_actors.csv')

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/Colab Notebooks/data_set/드라마 정보/data/mydramalistactors_tocsv.csv'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
actors_2.head()

In [ ]:
len(actors_2.person.unique())

In [ ]:
actors_3.head()

In [ ]:
len(actors_3.actor_name.unique())

In [ ]:
len(actors_2[actors_2.nationality == 'South Korean'].person.unique())

##서로 얼마나 일치하는지 확인
세 개 다 합치면 766개, 두 개 합치면 1712개.

일단 둘 다 돌려서 확인해보기.

생년은 촬영 당시 나이로 환산하기



In [ ]:
# ac_1 = actors_1.actor.copy()
# ac_1_pre = ac_1.str.lower().str.replace(' ','')

In [ ]:
ac_3 = actors_3.actor_name.copy()
ac_3_pre = ac_3.str.lower().str.replace(' ','')

In [ ]:
# #ac_1 + ac_3
# chck = pd.merge(ac_1_pre,ac_3_pre,left_on='actor',right_on='actor_name')
# len(chck['actor'].unique())

In [ ]:
ac_2 = actors_2[actors_2.nationality == 'South Korean'].person.copy()
ac_2_pre = ac_2.str.lower().str.replace(' ','')

In [ ]:
# #ac_1 + ac_2 + ac_3
# chck_2 = pd.merge(chck, ac_2_pre, how='left', left_on='actor',right_on='person')
# len(chck_2[chck_2.person.notnull()].person.unique())

##병합을 위한 전처리 (소문자 , 띄어쓰기 제외)

In [ ]:
# ac_1 = actors_1.copy()
# ac_1.kdrama_name = ac_1.kdrama_name.str.lower().str.replace(' ','')
# ac_1.actor = ac_1.actor.str.lower().str.replace(' ','')
# ac_1 = ac_1.rename(columns={'kdrama_name':'drama'})
# ac_1

In [ ]:
ac_2 = actors_2[actors_2.nationality =='South Korean'][['person','gender','dob']].copy()
ac_2.person = ac_2.person.str.lower().str.replace(' ','')
ac_2 = ac_2.rename(columns={'person':'actor'})
ac_2 = ac_2.drop_duplicates()

In [ ]:
ac_3 = actors_3[['actor_name','drama_name','role']].copy()
ac_3.actor_name = ac_3.actor_name.str.lower().str.replace(' ','')
ac_3.drama_name= ac_3.drama_name.str.lower().str.replace(' ','')
ac_3 = ac_3.rename(columns={'actor_name':'actor','drama_name':'drama'})
ac_3

##Merge (ac_2 + ac_3)
생각해보니까 ac_1가 딱히 필요 없음

ac_3처럼 역할 컬럼도 없는데...

그냥 손절~

ac_2, 3 이랑 합치니까 row 수 늘어남. 814개 개이득

그러므로 merge_1 과 ac_2를 비교하기

In [ ]:
mg_1 = pd.merge(ac_3, ac_2, how='inner', on='actor')
mg_1

In [ ]:
len(mg_1.actor.unique())

##또 다시 전처리(인코딩)

파생 변수 만드려고 하는데 아직 드라마 연도를 모름.

드라마 정보까지 합친 다음에 변환하기

인코딩은 그냥 모델링 과정에서 하는 게 더 안정적이다. 일단 패스!

In [ ]:
#태어난 연도만 뽑기

mg_1.dob = mg_1.dob.str.slice(start=-4)
mg_1 = mg_1.rename(columns={'dob' : 'birth_y'})

In [ ]:
mg_1

#드라마 데이터셋 만들기

##원본 데이터 확인
뭔가 1000개 넘개 데이터가 사라졌지만...
일단 레쓰고

나중에 더 보완할만한 데이터 있으면 찾아서 적용해보자


In [ ]:
drama_1 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/data_set/드라마 정보/data/kdramalist.csv')

In [ ]:
drama_1.info()

In [ ]:
drama_1.head()

In [ ]:
drama_1.columns

In [ ]:
d_1 = drama_1[['drama_name','Genres','start airing','Aired On','Original Network','score','scored by',]].copy()

In [ ]:
d_1_c = d_1.drama_name.str.lower().str.replace(' ','')

In [ ]:
test = pd.merge(mg_1.drama, d_1_c, left_on = 'drama',right_on='drama_name')
test

##전처리

In [ ]:
d_1.head()

In [ ]:
d_1.drama_name = d_1.drama_name.str.lower().str.replace(' ','')
d_1

In [ ]:
def clean_genres(genre_list):
  if isinstance(genre_list, list):
    return [genre.strip().replace("'", "") for genre in genre_list]
  return genre_list

d_1.Genres = d_1.Genres.apply(clean_genres)

In [ ]:
d_1.Genres[0] + d_1.Genres[1]

In [ ]:
d_1

In [ ]:
print(d_1.Genres)

In [ ]:
d_1.Genres.info()

In [ ]:
test = d_1.Genres[d_1.Genres.notnull()]

In [ ]:
k = []

for x in test:
  k.extend(x)

In [ ]:
k_1 = pd.DataFrame(k)
k_1[0].unique()

In [ ]:
k_1.value_counts()

In [ ]:
thriller = ['mystery','thriller','crime','psychological','horror','tokusatsu']
sf = ['fantasy','sci-fi','supernatural','adventure']
society = ['business','law','political','law','medical']
youth = ['youth','school','friendship']
hist_war = ['military','war','historical']
comedy = ['comedy','sitcom']
romance = ['romance','melodrama']
action = ['action','martial arts']
drama = ['drama','life','family']
etc = ['mature']

gen = [thriller,sf,society,youth,hist_war,comedy,romance,action,drama]


In [ ]:
for x in test:
  for y in x:
    for z in gen:
      if y in z:
        y = str(z)

test

In [ ]:
drama = d_1.dropna()

In [ ]:
#연도 변환
date_origin = drama['start airing']
date_clean = pd.to_datetime(date_origin, errors='coerce')
year = date_clean.dt.year


In [ ]:
drama['start airing'] = year

In [ ]:
drama

In [ ]:
#네트워크 변환
drama['Aired On'] = drama['Aired On'].str.slice(1, -1).str.replace("'","").str.split(',')

In [ ]:
drama['Original Network']=drama['Original Network'].str.slice(1,-1).str.replace("'","").str.upper()

NameError: name 'drama' is not defined

In [ ]:
drama['scored by'].describe()

In [ ]:
drama['score'].describe()

In [ ]:
#가중 평점 변환
m = 8500
drama['score_weighted'] = (drama['scored by'] / (drama['scored by'] + m)) * drama['score'] + (m / (drama['scored by']+m)) * drama['score'].mean()

In [ ]:
drama_p = drama.drop(columns=['score','scored by'])
drama_p.score_weighted=drama_p.score_weighted.round(2)
drama_p

In [ ]:
drama_p.to_csv('data_for_add')

In [ ]:
drama_p['Genres']= drama_p['Genres'].str.slice(1,-1).str.replace("'",'').str.split(',')

In [ ]:
drama_p['start airing'].max()

In [ ]:
g= drama_p['Genres']

In [ ]:
# name = ['thriller','sf','society','youth','hist_war','comedy','romance','action','drama', 'etc']  # 'etc' 포함

# g_list = []  # 결과 저장용

# for a in g:
#     n_list = []
#     for b in a:
#         matched = False
#         for x in name:
#             if b in dic[x]:
#                 if x not in n_list:
#                     n_list.append(x)
#                 matched = True
#         if not matched and b in ['food', 'music', 'sports']:
#             if b not in n_list:
#                 n_list.append(b)

#     g_list.append(n_list)  # 한 줄 결과 저장

# # 예시 출력
# for row in g_list[:10]:
#     print(row)


In [ ]:
drama_p['Original Network'] = drama_p['Original Network'].str.split(',')

In [ ]:
drama_p

#배우 + 드라마 데이터 병합

In [ ]:
DATA= pd.merge(mg_1, drama_p, left_on='drama', right_on='drama_name')
DATA

In [ ]:
DATA['birth_y'] = pd.to_numeric(DATA['birth_y'], errors='coerce')
DATA['age'] = DATA['start airing'] - DATA['birth_y'] + 1

In [ ]:
data_drama = DATA.drop(columns=['birth_y','drama_name','start airing'])

In [ ]:
data_drama

In [ ]:
thriller = ['mystery','thriller','crime','psychological','horror','tokusatsu']
sf = ['fantasy','sci-fi','supernatural','adventure']
society = ['business','law','political','law','medical']
youth = ['youth','school','friendship']
hist_war = ['military','war','historical']
comedy = ['comedy','sitcom']
romance = ['romance','melodrama']
action = ['action','martial arts']
drama = ['drama','life','family']
etc = ['mature']


name = ['thriller','sf','society','youth','hist_war','comedy','romance','action','drama']

dic = {'thriller':thriller, 'sf':sf, 'society':society, 'youth':youth, 'hist_war':hist_war,'comedy':comedy, 'romance':romance,'action':action,'drama': drama, 'etc' :etc}

In [ ]:
data_drama['Genres']

In [ ]:
def transform_genre(genre_list):
    """
    장르 리스트를 새로운 카테고리로 변환하는 함수 (공백 제거 기능 추가)
    """
    genre_map = {
        'thriller': ['mystery', 'thriller', 'crime', 'psychological', 'horror', 'tokusatsu'],
        'sf': ['fantasy', 'sci-fi', 'supernatural', 'adventure'],
        'society': ['business', 'law', 'political', 'medical'],
        'youth': ['youth', 'school', 'friendship'],
        'hist_war': ['military', 'war', 'historical'],
        'comedy': ['comedy', 'sitcom'],
        'romance': ['romance', 'melodrama'],
        'action': ['action', 'martial arts'],
        'drama': ['drama', 'life', 'family'],
        'etc': ['mature']
    }

    transformed_genres = set()
    keep_genres = ['food', 'music', 'sports']

    for genre in genre_list:
        # ✨ 이 부분이 핵심! 각 장르의 앞뒤 공백을 제거합니다.
        genre = genre.strip()

        if genre in keep_genres:
            transformed_genres.add(genre)
            continue

        for category, values in genre_map.items():
            if genre in values:
                transformed_genres.add(category)
                break

    return list(transformed_genres)

# --- 사용자 데이터로 테스트 ---
genres0 = ['romance', ' drama', ' melodrama', ' political']

# --- 함수 실행 및 결과 확인 ---
# sorted() 함수를 추가하여 결과를 항상 동일한 순서로 볼 수 있게 합니다.
transformed = sorted(transform_genre(genres0))
print(f"Original: {genres0} -> Transformed: {transformed}")

In [ ]:
for x in range(len(data_drama['Genres'])):
  data_drama['Genres'].loc[x] = transform_genre(data_drama['Genres'].loc[x])




In [ ]:
transform_genre(data_drama['Genres'].loc[0])

In [ ]:
data_drama

In [ ]:
data_drama.to_csv('data_drama_set')